# Clasificación de Dígitos MNIST - Versión Compatible

Versión actualizada para generar modelo compatible con versiones modernas de TensorFlow

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import math
import logging

logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print('TensorFlow versión:', tf.__version__)

In [ ]:
print('Cargando el dataset MNIST...')

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print('Dataset cargado exitosamente')

In [ ]:
def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

train_dataset = ds_train.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = ds_test.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)

print('Datos normalizados')

In [ ]:
print('Definiendo la estructura de la Red Neuronal...')

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

print('Modelo definido')
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print('Modelo compilado')

In [ ]:
BATCHSIZE = 32

train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(ds_info.splits['train'].num_examples)
train_dataset = train_dataset.batch(BATCHSIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.batch(BATCHSIZE)
test_dataset = test_dataset.cache()
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

print('Datos preparados para entrenamiento')

In [ ]:
print('Comenzando el entrenamiento del modelo (5 épocas)...')

history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=test_dataset,
    verbose=1
)

print('Entrenamiento completado')

In [ ]:
print('Evaluando el modelo...')

test_loss, test_accuracy = model.evaluate(
    test_dataset,
    verbose=1
)

print(f'Precisión en datos de prueba: {test_accuracy:.4f}')

In [ ]:
print('Guardando modelo compatible...')

model.save('modelo_MINST_completo.keras')

print('Modelo guardado como modelo_MINST_completo.keras')
print('Modelo compatible generado exitosamente!')

In [ ]:
print('Probando el modelo guardado...')

loaded_model = tf.keras.models.load_model('modelo_MINST_completo.keras')

loaded_test_loss, loaded_test_accuracy = loaded_model.evaluate(
    test_dataset,
    verbose=1
)

print(f'Precisión del modelo cargado: {loaded_test_accuracy:.4f}')
print('Modelo guardado y cargado correctamente!')